In [1]:
import pandas as pd
from glob import glob
from tqdm import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
csv_files = glob("data/*.csv")
len(csv_files)

43

#### Look up all the different columns of all kept files

In [3]:
columns = []
for f in tqdm(csv_files):
    df = pd.read_csv(f, encoding="latin-1")
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(" ", "")
    columns.append((set(df.columns), f))

100%|██████████| 43/43 [00:22<00:00,  1.92it/s]


In [4]:
unique_columns = []
files = []
for c, f in columns:
    if c not in unique_columns:
        unique_columns.append(c)
        files.append([f])
    else :
        i = unique_columns.index(c)
        files[i].append(f)
column_df = pd.DataFrame(list(zip(unique_columns, files)), columns=["Columns", "Files"])
column_df['num_files'] = column_df.Files.apply(len)
column_df = column_df.sort_values('num_files')

In [5]:
column_df

,Columns,Files,num_files
3,"{elementcode, yearcode, areacode, monthscode, ...",[data/Environment_Temperature_change_E_All_Dat...,1
4,"{elementcode, yearcode, itemcode, element, fla...",[data/Emissions_Agriculture_Energy_E_All_Data_...,1
1,"{elementcode, itemcode, element, flag, value, ...","[data/Environment_Energy_E_All_Data.csv, data/...",4
2,"{elementcode, yearcode, itemcode, areacode, no...",[data/Emissions_Agriculture_Burning_crop_resid...,4
0,"{elementcode, yearcode, itemcode, areacode, ar...",[data/Emissions_Agriculture_Enteric_Fermentati...,33


In [6]:
for i, x in column_df.Columns.iteritems():
    print(x)

{'elementcode', 'yearcode', 'areacode', 'monthscode', 'area', 'element', 'flag', 'value', 'unit', 'months', 'year'}
{'elementcode', 'yearcode', 'itemcode', 'element', 'flag', 'value', 'unit', 'country', 'countrycode', 'item', 'year'}
{'elementcode', 'itemcode', 'element', 'flag', 'value', 'unit', 'country', 'elementgroup', 'countrycode', 'item', 'year'}
{'elementcode', 'yearcode', 'itemcode', 'areacode', 'note', 'area', 'element', 'flag', 'value', 'unit', 'item', 'year'}
{'elementcode', 'yearcode', 'itemcode', 'areacode', 'area', 'element', 'flag', 'value', 'unit', 'item', 'year'}


#### Get mapping of elementcode => (element, unit) and itemcode => item

In [7]:
from mappings import get_all_element_mapping
data_dir = "data/"
element_mapping = get_all_element_mapping(data_dir)

Fount 43 files in data/


In [8]:
from mappings import get_all_item_mapping
item_mapping = get_all_item_mapping(data_dir)

Fount 43 files in data/
Item Code not found in data/Environment_Temperature_change_E_All_Data_(Normalized).csv


#### Print out the description of each csv (**DO NOT RUN**)

In [181]:
from mappings import get_item_mapping, get_element_mapping
from utils import rename_columns
import os


In [182]:
with open("csv_description.md", 'w+') as f:
    for file in csv_files:
        df = rename_columns(pd.read_csv(file, encoding="latin-1"))
        f.write("#### " + os.path.basename(file) + "\n")
        if 'itemcode' in df.columns:
            item_mapping = get_item_mapping(df)
        else:
            item_mapping = pd.DataFrame([])
        element_mapping = get_element_mapping(df)
        f.write("Items : \n" + item_mapping.to_string(index=False) + "\n\n")
        f.write("Elements : \n" + element_mapping.to_string(index=False) + "\n\n")
        f.write("\n\n ------\n\n")